In [10]:
import { ChatOpenAI } from "@langchain/openai";
import { HumanMessage } from "@langchain/core/messages";
import { StringOutputParser, StructuredOutputParser, CommaSeparatedListOutputParser, } from "@langchain/core/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { load } from "dotenv";
let env = await load();
let process = { env };

let model = new ChatOpenAI({
    configuration: {
        baseURL: process.env.baseURL,
    },
});


String Output Parset 获取文本内容

In [2]:
let parser = new StringOutputParser();
let chain = model.pipe(parser);
await chain.invoke([new HumanMessage("给我讲个笑话")]);

"为什么鸡会过马路？\n因为它想证明自己不是只会在鸡笼里走！哈哈哈！"

StructuredOutputParser 引导模型以你需要的格式进行输出

In [3]:
let parser = StructuredOutputParser.fromNamesAndDescriptions({
    answer: "用户问题的答案",
    evidence: "你回答用户问题所依据的答案",
    confidence: "问题答案的可信度评分，格式是百分数",
});
let prompt = PromptTemplate.fromTemplate("尽可能的回答用户的问题\n{instructions} \n{question}");
let chain = prompt.pipe(model).pipe(parser);
let res = await chain.invoke({
    question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
    instructions: parser.getFormatInstructions(),
});
console.log(res);


{
  answer: "莱昂纳多·达·芬奇是蒙娜丽莎的作者。",
  evidence: "根据艺术历史记录和文献记载",
  confidence: "95%"
}


List Output Parser 数组格式输出

In [4]:
let parser = new CommaSeparatedListOutputParser();
console.log(parser.getFormatInstructions());
let prompt = PromptTemplate.fromTemplate("列出一些{name}喜欢的食物\n{instructions}");
let chain = prompt.pipe(model).pipe(parser)
let res = await chain.invoke({
  name:'周恩来',
  instructions:parser.getFormatInstructions()
})
console.log(res)


Your response should be a list of comma separated values, eg: `foo, bar, baz`
[ "鱼香肉丝", "啤酒", "芝麻糖", "盐水鸭", "包子" ]


Auto Fix Parser 输出验证

In [13]:
import { z } from "zod";

let schema = z.object({
    answer: z.string().describe("用户问题的答案"),
    confidence: z.number().min(0).max(100).describe("问题答案的可信度评分，满分100分"),
});

let parser = StructuredOutputParser.fromZodSchema(schema);
let prompt = PromptTemplate.fromTemplate("尽可能的回答用户的问题\n{instructions} \n{question}");
let chain = prompt.pipe(model).pipe(parser);
let res = await chain.invoke({
    question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
    instructions: parser.getFormatInstructions(),
});
console.log(res);

{ answer: "莱昂纳尔达芬奇", confidence: 95 }


In [23]:
import { OutputFixingParser } from "langchain/output_parsers";
// outputfix修复出错输出
let wrongOutput = {
  "answer": "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
  "sources": "90%" 
};

let fixParser = OutputFixingParser.fromLLM(model, parser);
let output = await fixParser.parse(JSON.stringify(wrongOutput));


SyntaxError: Identifier 'fixParser' has already been declared